In [1]:
from pyspark import SparkContext
import numpy as np

## rdd-1
Two ways to create RDDs-parallelizing an exiting collection in driver program or referencing dataset in external storage system, such as shared file-syste, HDFS, HBase, or any data source offering a Hadoop InputFormat.

For illustration with a python based approach, first type example -- we create a simple Python array of 20 random integers (between 0 and 10) and create and RDD object.

In [2]:
sc = SparkContext(master="local[4]")

In [3]:
lst = np.random.randint(0,10,20)
A = sc.parallelize(lst)

Note the '4' in the arguement. It denotes 4 computing cores (in your local machine) to be used for the SparkContext object. We can check the thype of the RDD object, we get the following.

In [4]:
type(A)

pyspark.rdd.RDD

Opposite to parrallelization is the collection(with collect()) which brings all the distributed elements and returns them to the head node.

In [5]:
A.collect()

[0, 2, 8, 7, 5, 7, 2, 4, 9, 3, 1, 1, 1, 1, 8, 7, 4, 0, 9, 8]

But A is no longer a simple Numpy array. We can use the glom() method to check how the partitions are created.

In [6]:
A.glom().collect()

[[0, 2, 8, 7, 5], [7, 2, 4, 9, 3], [1, 1, 1, 1, 8], [7, 4, 0, 9, 8]]

Now stop the SC and reninitialized it with 2 cores and see what happens when you repeat the process.

In [7]:
sc.stop()

In [8]:
sc = SparkContext(master="local[2]")

In [9]:
A = sc.parallelize(lst)

In [10]:
A.glom().collect()

[[0, 2, 8, 7, 5, 7, 2, 4, 9, 3], [1, 1, 1, 1, 8, 7, 4, 0, 9, 8]]

Here RDD is now distributed over two chunks, not four!

The above is the first step in distributed data analytics i.e. controlling how your data is partitioned over smaller chunks for further processing.

## Some Examples of Basic Operations with RDD & PySpark

In [11]:
A.count()

20

In [12]:
A.first()

0

In [13]:
A.take(5)

[0, 2, 8, 7, 5]

## Removing duplicates with using distinct


NOTE: This operation requires a shuffle in order to detect duplication across partitions. So, it is a slow operation. Donot overdo it.

In [14]:
A_distinct = A.distinct()

In [15]:
A_distinct.collect()

[0, 2, 8, 4, 7, 5, 9, 3, 1]

### To sum all the elements use reduce method

Note the use of lambda function in this,

In [16]:
A.reduce(lambda x, y: x+y)

87

### OR the direct sum() method

In [17]:
A.sum()

87

### Finding maximum element by reduce

In [19]:
A.reduce(lambda x,y: x if x > y else y)

9

### Finding longest word in a blob of text

In [20]:
words = 'These are some of the best Machintosh computers ever'.split(' ')


In [21]:
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v : w if len(w) > len(v) else v)

'Machintosh'

### Use filter for logic-based filtering

In [22]:
#Return RDD with elements (greater than zero) divisible by 3
A.filter(lambda x:x%3 == 0 and x!=0).collect()

[9, 3, 9]

### Regular Python Functions to use with reduce()

In [23]:
def largerThan(x,y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x) > len(y):
        return x
    elif len(y) > len(x):
        return y
    else:
        if x < y: return x
        else: return y

In [24]:
wordRDD.reduce(largerThan)

'Machintosh'

Note here the x < y does the lexicographic comparison and determines that Macintosh is larger than computers!

### Mapping operation with a lambda function with PySpark

In [25]:
B = A.map(lambda x:x*x)
B.collect()

[0, 4, 64, 49, 25, 49, 4, 16, 81, 9, 1, 1, 1, 1, 64, 49, 16, 0, 81, 64]

### Mapping with a regular Python funtion in PySpark

In [26]:
def square_if_odd(x):
    """
    Squares if odd, otherwise keeps the arguement unchanged
    """
    if x%2==1:
        return x*x
    else:
        return x

In [27]:
A.map(square_if_odd).collect()

[0, 2, 8, 49, 25, 49, 2, 4, 81, 9, 1, 1, 1, 1, 8, 49, 4, 0, 81, 8]

### groupby returns a RDD of grouped elements (iterable) as per a given group operation

We will use a list-comprehension along with the groupby to create a list of two elements, each having a header (Tthe result of the lambda function, simple modulo 2 here), and a sorted list of the elements which gave rise to that result. You can imagine easily this kind of seperation can come particularly handy for processing data which needs to be binned/canned out based on particular operation performed over them.

In [28]:
result = A.groupBy(lambda x:x%2).collect()

In [32]:
result

[(0, <pyspark.resultiterable.ResultIterable at 0x7f915071b400>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f915071b6d8>)]

In [29]:
sorted([(x,sorted(y)) for (x,y) in result])

[(0, [0, 0, 2, 2, 4, 4, 8, 8, 8]), (1, [1, 1, 1, 1, 3, 5, 7, 7, 7, 9, 9])]

### Using histogram

The histogram() method takes a list of bins/buckets and returns a tuple with result of the histogram(binning),

In [33]:
B.histogram([x for x in range(0,100,10)])

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [9, 2, 1, 0, 3, 0, 3, 0, 2])

### Lazy evaluation with PySpark (and Caching)

Lazy evaluation is an evaluation/computation strategy which prepares a detailed step-by-step internal map of the execution piplines for computing task, but delays the final execution until when it is absolutely needed. This strategy is at the heart of Spark for speeding up many parallelized Big Data operations.

In [36]:
sc.stop()

In [37]:
#let us use two CPU cores
scc = SparkContext(master="local[2]")

### Make a RDD with 1 million elements

In [41]:
%%time
rdd1 = scc.parallelize(range(1000000))

CPU times: user 0 ns, sys: 2.23 ms, total: 2.23 ms
Wall time: 5.19 ms


### Some computing function - taketime

In [42]:
from math import cos

In [43]:
def taketime(x):
    [cos(j) for j in range(100)]
    return cos(x)

### Check how much time is taken by taketime function

In [44]:
%%time
taketime(2)

CPU times: user 25 µs, sys: 5 µs, total: 30 µs
Wall time: 32.2 µs


-0.4161468365471424

Remember this result, the taketime() function took a wall time of 32.2us. Of course, the exact number will depend on the machine you are working on.

### Now do the map operation of the function

In [45]:
%%time
interim = rdd1.map(lambda x: taketime(x))

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 17.4 µs


How come each taketime function takes 45.8 us but the map operation with a 1 million elements RDD also took similar time?

Because of lazy evaluation i.e. nothing was computed in the previous step, just a plan of execution was made. The variable interim does not point to a data structure, instead it points to a plan of execution, expressed as a dependency graph. The dependency graph defines how RDDs are computed from each other.

### The actual execution by reduce method

In [46]:
%%time
print('output =', interim.reduce(lambda x, y: x+y))

output = -0.2887054679684353
CPU times: user 5.43 ms, sys: 3.69 ms, total: 9.12 ms
Wall time: 5.69 s


So, the wall time here is 5.69 seconds. Remember, the taketime() function had a wall time of 32.2 us? Therefore, we expect the total time to be on the order of 13 seconds for a 1-million array. Because of parallel operation on two cores, it took only 5.69 seconds.

Now we have not saved any intermediate results in interim, so another simple opeartion (e.g. counting elements >0) will take almost some time.

In [47]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 8.04 ms, sys: 1.53 ms, total: 9.57 ms
Wall time: 5.17 s


### Caching to reduce computation time on similar operation (spending memory)

Remember the dependency graph that we built in the previous step? We can run the same computation as before with cache method to tell the dependency graph to plan for caching.

In [48]:
%%time
interim = rdd1.map(lambda x: taketime(x)).cache()

CPU times: user 3.68 ms, sys: 548 µs, total: 4.23 ms
Wall time: 13.4 ms


The first computation will not improve, but it caches the interim result,

In [49]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.2887054679684353
CPU times: user 5.8 ms, sys: 2.23 ms, total: 8.03 ms
Wall time: 5.5 s


Now run the same filter method with the help of cached result,

In [50]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 4.19 ms, sys: 4.59 ms, total: 8.77 ms
Wall time: 209 ms


Wow! The compute time came down to less second from earlier! This way, caching and parallelization with lazy excution, is the core feature of programming with Spark.